In [1]:
import numpy as np
import tensorflow as tf
from utils import *
from tqdm import tqdm
from matplotlib import pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data as mnist

In [2]:
data = mnist.read_data_sets('./data/', one_hot=True, reshape=False)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./data/t10k-images-idx3-ubyte.gz
Extracting ./data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
train_img = data.train.images
train_lab = data.train.labels

test_img = data.train.images
test_lab = data.train.labels

In [4]:
def lenet(X, w, scope = 'LeNet'):
    with tf.variable_scope(scope, 'LeNet', [X, 10]):
        conv1 = conv2d(X, w['w1'])
        act1 = tf.nn.elu(conv1)
        
        pool1 = max_pool_2d(act1)
        conv2 = conv2d(pool1, w['w2'])
        act2 = tf.nn.elu(conv2)
        
        pool2 = max_pool_2d(act2)
        
        gap = tf.reduce_mean(pool2, (1,2))
        
        logits = tf.matmul(gap, w['w3'])
        
        return logits, pool2

In [5]:
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])

In [6]:
w = {
    'w1':init_w('w1', [5,5,1,32]),
    'w2':init_w('w2', [5,5,32,64]),
    'w3':init_w('w3', [64,10])
    }

In [7]:
logit, act = lenet(X, w)

In [8]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=logit))
train_step = tf.train.GradientDescentOptimizer(0.001).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(logit, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [9]:
#config = tf.ConfigProto()
#config.gpu_options.allow_growth = True
#sess = tf.Session(config=config)
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [14]:
batch = 1000
tot_batch = int(len(train_img)/batch)+1

for i in tqdm(range(10000)):
    acc = 0.
    cost = 0.
    for j in range(tot_batch):
        batch_xs, batch_ys= data.train.next_batch(batch)
        sess.run(train_step, feed_dict={X: batch_xs, Y: batch_ys})
        acc = acc + sess.run(accuracy, feed_dict={X: batch_xs, Y: batch_ys})
        cost = cost + sess.run(cross_entropy, feed_dict={X: batch_xs, Y: batch_ys})
        
    if i+1 == 100:
        print('Epoch : ', i+1, 'Cost : ', cost/tot_batch, 'Accuracy : ', acc/tot_batch)

  1%|          | 100/10000 [03:23<5:34:58,  2.03s/it]

Epoch :  100 Cost :  0.609418544386 Accuracy :  0.837696429874


  1%|          | 110/10000 [03:43<5:34:39,  2.03s/it]

KeyboardInterrupt: 

In [185]:
num = 25
t_logit, t_act = sess.run([logit, act], feed_dict={X:test_img[num:num+2]})

In [104]:
w_act = sess.run(w['w3'])

In [132]:
w_act.shape

(64, 10)

In [224]:
def cam(act, w, size):
    out_cha = int(act.shape[-1])
    resized = tf.image.resize_bilinear(act,[size,size])
    resized = tf.reshape(resized, [-1, out_cha])
    
    out = None
    for i in range(10):
        label_w = tf.gather(tf.transpose(w['w3']), i)
        label_w = tf.reshape(label_w, [out_cha,1])

        tmp_out = tf.matmul(resized, label_w)
        tmp_out = tf.reshape(tmp_out, [-1, size, size, 1])
        if i==0:
            out = tmp_out
        else : 
            out = tf.concat((out, tmp_out), axis = 3)
    return out

In [225]:
out = cam(t_act, w, 28)